In [11]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import numpy as np
import os
from tqdm import tqdm
 
file = 'BBB'   # NA、NT、NC

file_prefixes = ["smiles","DMPNN", "ecfp", "mfbert", "padel"]
file_extension = ".csv"

file_prefix = f"{file}_Feature_fusion/data_train"
file_extension = ".csv"

val_file_prefix = f"{file}_Feature_fusion/data_valid"
val_file_extension = ".csv"

start_index = 0
end_index = 4

auc_scores = []
acc_scores = []
f1_scores = []
recall_scores = []
mcc_scores = []


for i in tqdm(range(start_index, end_index + 1), desc="Processing files"):  # 使用 tqdm 添加进度条  
    combined_data_train = None
    combined_data_valid = None
    combined_data_test = None
    for file_feature in file_prefixes:
        #从randomi/中获取train，valid，test    
        
        file_path_train = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "train" + file_extension
        data_train = pd.read_csv(file_path_train) 
                
        file_path_valid = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "valid" + file_extension
        data_valid = pd.read_csv(file_path_valid) 

        file_path_test = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "test" + file_extension
        data_test = pd.read_csv(file_path_test)   
        
        data_train = pd.read_csv(file_path_train)     
        data_valid = pd.read_csv(file_path_valid)   
        data_test  = pd.read_csv(file_path_test) 
        
        if combined_data_train is None:
            combined_data_train = data_train
        if combined_data_valid is None:
            combined_data_valid = data_valid
        if combined_data_test is None:   
            combined_data_test  = data_test
        else:
            combined_data_train = pd.concat([combined_data_train, data_train], axis=1)     
            combined_data_valid = pd.concat([combined_data_valid, data_valid], axis=1) 
            combined_data_test = pd.concat([combined_data_test, data_test], axis=1) 
    
    folder_path = f'{file}_Feature_fusion'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    combined_data_train.to_csv(f'{file}_Feature_fusion/data_train{i}.csv', index=False)
    combined_data_valid.to_csv(f'{file}_Feature_fusion/data_valid{i}.csv', index=False)
    combined_data_test.to_csv(f'{file}_Feature_fusion/data_test{i}.csv', index=False)


Processing files: 100%|██████████| 5/5 [01:08<00:00, 13.73s/it]


In [12]:
# 读取CSV文件
file_prefix = f"{file}_Feature_fusion/data_train"
file_extension = ".csv"

val_file_prefix = f"{file}_Feature_fusion/data_valid"
val_file_extension = ".csv"

for i in range(start_index, end_index+1):
    
    # 读取数据
    file_path = file_prefix + str(i) + file_extension
    data = pd.read_csv(file_path)
    null_indices = None  # 初始化 null_indices 为 None
    if data.isnull().values.any():  # 判断是否有空值
        print("数据中存在空值，空值坐标为：")
        null_indices = np.where(data.isnull())  # 获取空值的坐标
        if null_indices is not None:  # 再次判断 null_indices 是否有值
            for row_index, col_index in zip(null_indices[0], null_indices[1]):
                print(f"行 {row_index}，列 {col_index}")
    else:
        print("数据中没有空值")
    print(f"该文件的行数为：{data.shape[0]}，列数为：{data.shape[1]}")
    

数据中没有空值
该文件的行数为：10320，列数为：2828
数据中没有空值
该文件的行数为：10320，列数为：2828
数据中没有空值
该文件的行数为：10320，列数为：2828
数据中没有空值
该文件的行数为：10320，列数为：2828
数据中没有空值
该文件的行数为：10320，列数为：2828


In [16]:
# 读取CSV文件
for i in range(start_index, end_index+1):
    
    # 读取数据
    file_path = file_prefix + str(i) + file_extension
    data = pd.read_csv(file_path)
    
    # 提取特征和标签
    x_input = data.drop(['SMILES'], axis=1).values
    y_output = data['active'].values
    
        # 遍历数据，将 inf 赋值为 0
    x_input = np.nan_to_num(x_input, posinf=0, neginf=0)
    
#        # 检查并输出非 float64 数据的坐标
#    for row_index, row in enumerate(x_input):
#        for col_index, value in enumerate(row):
#            if x_input.dtype!= np.float64:
#                print(f"非 float64 数据位于 ({row_index}, {col_index})，值为: {value}")
#    
    #print('x_input',x_input)
    #print('y_output',y_output)
    
    
    val_x_input = data.drop(['SMILES'], axis=1).values
    val_y_output = data['active'].values
    
    val_x_input = np.nan_to_num(x_input, posinf=0, neginf=0)
    
    # 转换为Tensor
    print(x_input.dtype)
#    x = torch.Tensor(x_input)
#    y_true = torch.Tensor(y_output).view(-1, 1)    
#    
#    val_x = torch.Tensor(val_x_input)
#    val_y_true = torch.Tensor(val_y_output).view(-1, 1)
    
#    # 查找 x_input 中的无穷值位置
#    infinity_positions_x = np.argwhere(np.isinf(x_input))
#    if len(infinity_positions_x) > 0:
#        column_names = data.drop(['SMILES'], axis=1).columns
#        infinity_column_names_x = [column_names[pos[1]] for pos in infinity_positions_x]
#        print("在 x_input 中的无穷值所在列名：", infinity_column_names_x, '         位置：',infinity_positions_x)
#    else:
#        print("在 x_input 中没有无穷值")
#
#    # 查找 val_x_input 中的无穷值位置
#    infinity_positions_val_x = np.argwhere(np.isinf(val_x_input))
#    if len(infinity_positions_val_x) > 0:
#        column_names = data.drop(['SMILES'], axis=1).columns
#        infinity_column_names_val_x = [column_names[pos[1]] for pos in infinity_positions_val_x]
#        print("在 val_x_input 中的无穷值所在列名：", infinity_column_names_val_x, '        位置：',infinity_positions_x)
#    else:
#        print("在 val_x_input 中没有无穷值")
    
        # 标准化特征（假设特征是数值型的）
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    x_input = scaler.fit_transform(x_input)
    val_x_input = scaler.fit_transform(val_x_input)

    # 数据放缩处理（例如，将特征值缩放到 0 到 1 之间）
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    x_input = min_max_scaler.fit_transform(x_input)
    val_x_input = min_max_scaler.fit_transform(val_x_input)
    
    x = torch.Tensor(x_input)
    y_true = torch.Tensor(y_output).view(-1, 1)    
    
    val_x = torch.Tensor(val_x_input)
    val_y_true = torch.Tensor(val_y_output).view(-1, 1)
    
    # 数据维度
    input_dim = x.shape[1]
    output_dim = 1
    hidden_dim1 = 256
    hidden_dim2 = 128
    hidden_dim3 = 64
    learning_rate = 0.001
    num_epochs = 50
    weight_decay = 0.001
    
    # MLP模型定义
    mlp = nn.Sequential(  
        nn.Linear(input_dim, hidden_dim1),  # 第一层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim1, hidden_dim2),  # 第二层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim2, hidden_dim3),  # 第三层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim3, output_dim),  # 输出层  
        nn.Sigmoid()  
    )  
        
    # 优化器和损失函数
    optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate,weight_decay=weight_decay)
    loss_func =  nn.BCEWithLogitsLoss()
    #loss_func =  nn.MSELoss()
    best_val_loss = float('inf')
    num_iterations_without_improvement = 0  # Initialize counter for iterations without improvement. 
    
    for epoch in range(num_epochs):
        
        print(f"开始训练第{epoch}组epochs")
        prediction = mlp(x)
        #print('prediction',prediction)
        
        #prediction_np = prediction.detach().numpy()
        #df = pd.DataFrame(prediction_np)
        #df.to_csv('prediction.csv', index=False)
        
        loss = loss_func(prediction, y_true)
        print('loss',loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(optimizer)
        
        with torch.no_grad():
            val_prediction=mlp(val_x)
            val_loss = loss_func(val_prediction, val_y_true)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item()}, Val Loss: {val_loss.item()}")
        
        #print("检查数据和模型输出:")
        #print("是否存在 NaN 或无穷大值在训练预测结果中:", torch.isnan(prediction).any() or torch.isinf(prediction).any())
        #print("是否存在 NaN 或无穷大值在验证预测结果中:", torch.isnan(val_prediction).any() or torch.isinf(val_prediction).any())
        #print("是否存在 NaN 或无穷大值在训练真实值中:", torch.isnan(y_true).any() or torch.isinf(y_true).any())
        #print("是否存在 NaN 或无穷大值在验证真实值中:", torch.isnan(val_y_true).any() or torch.isinf(val_y_true).any())
        #
          
        if val_loss < best_val_loss:
            best_val_loss = val_loss  
            num_iterations_without_improvement = 0
            torch.save(mlp.state_dict(), f"./models/model_{i}.pt")
            #torch.save(mlp, "model_saved/mlp1.pt")
            print(f'model_{i}训练结果更新为第{epoch}个模型')
            best_epoch = epoch
        else :
            num_iterations_without_improvement += 1  
            if num_iterations_without_improvement == 30:
                print('30次迭代没有更新，结束迭代')
                #print(f'最棒的模型是第{best_epoch}个epoch')
                break
        print('         ')
        

    prediction_np = prediction.data.numpy()
        # 将 prediction 添加到 DataFrame
    #data['prediction'] = prediction_np
        # 将 DataFrame 保存到 CSV 文件
    #data.to_csv('output/output_train.csv', index=True)
    
    # 预测并计算AUC
    prediction_np = prediction.detach().numpy().flatten()
    auc = roc_auc_score(y_true, prediction_np)
    
    threshold = 0.5
    binary_prediction = np.where(prediction_np > threshold, 1, 0)
    
    acc = accuracy_score(y_true, binary_prediction)
    f1 = f1_score(y_true, binary_prediction)
    recall = recall_score(y_true, binary_prediction)
    mcc = matthews_corrcoef(y_true, binary_prediction)
    output_file = "train_scores.csv"
    
    
    auc_scores.append(auc)
    acc_scores.append(acc)
    f1_scores.append(f1)
    recall_scores.append(recall)
    mcc_scores.append(mcc)
    
    print('第',i,'组模型')
    print("AUC:", auc)
    print("ACC:", acc)
    print("F1:", f1)
    print("Recall:", recall)
    print("MCC:", mcc)
    print('    ')
      
    

float64
开始训练第0组epochs
loss tensor(0.6611, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 1/50, Train Loss: 0.6611285209655762, Val Loss: 0.6593686938285828
model_0训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.6594, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 2/50, Train Loss: 0.6593686938285828, Val Loss: 0.655245304107666
model_0训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.6552, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 3/50, Train Loss: 0.655245304107666, Val Loss: 0.6498987674713135
model_0训练结果更新为第2个模型
         
开始训练第3组epochs
loss tensor(0.6499, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 4/50, Train Loss: 0.6498987674713135, Val Loss: 0.6427257061004639
model_0训练结果更新为第3个模型
         
开始训练第4组epochs
loss tensor(0.6427, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 5/50, Train Loss: 0.6427257061004639, Val Loss: 0.6337592601776123
model_0训练结果更新为第4个模型
         
开始训练第5组epochs
loss tensor(0.6338, grad_fn=<BinaryCrossEntropyWithLogi

Epoch 47/50, Train Loss: 0.4654705226421356, Val Loss: 0.4645395576953888
model_0训练结果更新为第46个模型
         
开始训练第47组epochs
loss tensor(0.4645, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 48/50, Train Loss: 0.4645395576953888, Val Loss: 0.4655628800392151
         
开始训练第48组epochs
loss tensor(0.4656, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 49/50, Train Loss: 0.4655628800392151, Val Loss: 0.46471908688545227
         
开始训练第49组epochs
loss tensor(0.4647, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 50/50, Train Loss: 0.46471908688545227, Val Loss: 0.4644738435745239
model_0训练结果更新为第49个模型
         
第 0 组模型
AUC: 0.9971453961492245
ACC: 0.9817829457364341
F1: 0.9853331252925573
Recall: 0.9780083630168809
MCC: 0.9614896512821223
    
float64
开始训练第0组epochs
loss tensor(0.6636, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 1/50, Train Loss: 0.6636444330215454, Val Loss: 0.6600914001464844
model_1训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.6601, grad_fn=<

Epoch 41/50, Train Loss: 0.46426627039909363, Val Loss: 0.4644860625267029
         
开始训练第41组epochs
loss tensor(0.4645, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 42/50, Train Loss: 0.4644860625267029, Val Loss: 0.4650534391403198
         
开始训练第42组epochs
loss tensor(0.4651, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 43/50, Train Loss: 0.4650534391403198, Val Loss: 0.4652916193008423
         
开始训练第43组epochs
loss tensor(0.4653, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 44/50, Train Loss: 0.4652916193008423, Val Loss: 0.46384409070014954
model_1训练结果更新为第43个模型
         
开始训练第44组epochs
loss tensor(0.4638, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 45/50, Train Loss: 0.46384409070014954, Val Loss: 0.4641015827655792
         
开始训练第45组epochs
loss tensor(0.4641, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 46/50, Train Loss: 0.4641015827655792, Val Loss: 0.4646904170513153
         
开始训练第46组epochs
loss tensor(0.4647, grad_fn=<BinaryCrossEn

Epoch 36/50, Train Loss: 0.465215265750885, Val Loss: 0.4650224447250366
model_2训练结果更新为第35个模型
         
开始训练第36组epochs
loss tensor(0.4650, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 37/50, Train Loss: 0.4650224447250366, Val Loss: 0.4648529291152954
model_2训练结果更新为第36个模型
         
开始训练第37组epochs
loss tensor(0.4649, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 38/50, Train Loss: 0.4648529291152954, Val Loss: 0.46469613909721375
model_2训练结果更新为第37个模型
         
开始训练第38组epochs
loss tensor(0.4647, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 39/50, Train Loss: 0.46469613909721375, Val Loss: 0.46454155445098877
model_2训练结果更新为第38个模型
         
开始训练第39组epochs
loss tensor(0.4645, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 40/50, Train Loss: 0.46454155445098877, Val Loss: 0.46439194679260254
model_2训练结果更新为第39个模型
         
开始训练第40组epochs
loss tensor(0.4644, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 41/50, Train Loss: 0.46439194679260254, Val Loss: 

Epoch 30/50, Train Loss: 0.4671134948730469, Val Loss: 0.466581791639328
model_3训练结果更新为第29个模型
         
开始训练第30组epochs
loss tensor(0.4666, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 31/50, Train Loss: 0.466581791639328, Val Loss: 0.4661474823951721
model_3训练结果更新为第30个模型
         
开始训练第31组epochs
loss tensor(0.4661, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 32/50, Train Loss: 0.4661474823951721, Val Loss: 0.4658007323741913
model_3训练结果更新为第31个模型
         
开始训练第32组epochs
loss tensor(0.4658, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 33/50, Train Loss: 0.4658007323741913, Val Loss: 0.46552300453186035
model_3训练结果更新为第32个模型
         
开始训练第33组epochs
loss tensor(0.4655, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 34/50, Train Loss: 0.46552300453186035, Val Loss: 0.4653048813343048
model_3训练结果更新为第33个模型
         
开始训练第34组epochs
loss tensor(0.4653, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 35/50, Train Loss: 0.4653048813343048, Val Loss: 0.465

Epoch 24/50, Train Loss: 0.472797155380249, Val Loss: 0.4717417061328888
model_4训练结果更新为第23个模型
         
开始训练第24组epochs
loss tensor(0.4717, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 25/50, Train Loss: 0.4717417061328888, Val Loss: 0.47082704305648804
model_4训练结果更新为第24个模型
         
开始训练第25组epochs
loss tensor(0.4708, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 26/50, Train Loss: 0.47082704305648804, Val Loss: 0.4700251519680023
model_4训练结果更新为第25个模型
         
开始训练第26组epochs
loss tensor(0.4700, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 27/50, Train Loss: 0.4700251519680023, Val Loss: 0.46931716799736023
model_4训练结果更新为第26个模型
         
开始训练第27组epochs
loss tensor(0.4693, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 28/50, Train Loss: 0.46931716799736023, Val Loss: 0.46868735551834106
model_4训练结果更新为第27个模型
         
开始训练第28组epochs
loss tensor(0.4687, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch 29/50, Train Loss: 0.46868735551834106, Val Loss: 

In [17]:
te_file_prefix = f"{file}_Feature_fusion/data_test"
te_file_extension = ".csv"

import os 

avg_auc = 0.0  
avg_acc = 0.0  
avg_f1 = 0.0  
avg_recall = 0.0  
avg_mcc = 0.0  


#te_file_paths = [te_file_prefix + str(i) + te_file_extension for i in range(start_index, end_index+1)]  
results = pd.DataFrame(columns=['AUC', 'Accuracy', 'F1', 'Recall', 'MCC'])   
    
#for i,te_file_path in enumerate(te_file_paths):  
for i in range(start_index, end_index): 
    te_file_path = te_file_prefix + str(i) + te_file_extension
    data_test = pd.read_csv(te_file_path)  
    if file == 'BBB':
        data_test = data_test.head(129)  

    print(data_test)
      
    te_x_input = data_test.drop(['SMILES'], axis=1).values  
    #print('1te_x_input.shape',te_x_input.shape)
    
    te_y_true = data_test['active'].values  
    te_x_input = te_x_input.astype(np.float64)
    #print('2te_x_input.shape',te_x_input.shape)
    
    te_x_input = np.nan_to_num(te_x_input, posinf=0, neginf=0)
    #print('3te_x_input',te_x_input.shape)    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    te_x_input = scaler.fit_transform(te_x_input)

#    # 数据放缩处理（例如，将特征值缩放到 0 到 1 间）
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    te_x_input = min_max_scaler.fit_transform(te_x_input)
    te_x_input = torch.Tensor(te_x_input)
    print('43te_x_input.shape',te_x_input.shape)
    mlp.load_state_dict(torch.load(f"models/model_{i+1}.pt")) 
    
    
#    print("Length of te_prediction:", len(te_prediction))
#    print("Length of data_test:", len(data_test))
#    print("Length of data index:", len(data.index))
    
    with torch.no_grad():    
        
       # print("47  te_x_input  te_x_input",te_x_input)
    
        #print('5te_x_input.shape',te_x_input.shape)
        te_prediction = mlp(te_x_input)  
        
        ############################################################
        te_prediction = te_prediction.numpy()
        data = pd.DataFrame(te_prediction)
        data.to_csv(f'output1/test_prediction{i+1}.csv', mode='a', index=False)
        ############################################################
        
        #print('te_y_true',len(te_y_true))
        #print('te_prediction',len(te_prediction))
        #print('te_y_true',te_y_true)
        #print('te_prediction',te_prediction)
        #print('检查 te_y_true 是否包含 NaN',np.isnan(te_y_true).any())  # 检查 te_y_true 是否包含 NaN
        #print('检查 te_y_true 是否包含无穷值',np.isinf(te_y_true).any())  # 检查 te_y_true 是否包含无穷值
        #print('检查 te_prediction 是否包含 NaN',np.isnan(te_prediction).any())  # 检查 te_prediction 是否包含 NaN
        #print('检查 te_prediction 是否包含无穷值',np.isinf(te_prediction).any())  # 检查 te_prediction 是否包含无穷值
        #
        auc = roc_auc_score(te_y_true, te_prediction)
        #print(auc)
        
        #print('54  te_predictionte_prediction',te_prediction)    
        #te_prediction = te_prediction.numpy()      
        data_test[f'te_prediction_{i+1}'] = te_prediction 
        
        auc = roc_auc_score(te_y_true, te_prediction)
        #print(auc)
        
        #data_test.to_csv(f'output1/te_prediction{i+1}.csv', mode='a', index=True)
        #print('te_prediction  ',te_prediction)
        
        auc = roc_auc_score(te_y_true, te_prediction)  
        print(f'AUC_{i+1}: {auc}')
        
        te_prediction_binary = np.where(te_prediction > 0.5, 1, 0)  
        acc = accuracy_score(te_y_true, te_prediction_binary)  
        print(f'Accuracy_{i+1}: {acc}')
        f1 = f1_score(te_y_true, te_prediction_binary)
        recall = recall_score(te_y_true, te_prediction_binary)
        mcc = matthews_corrcoef(te_y_true, te_prediction_binary)
        print(f'f1_{i+1}: {f1}')
        print(f'recall_{i+1}: {recall}')
        print(f'mcc_{i+1}: {mcc}')
        print('             ')
        print('             ')
        
        avg_auc += auc  
        avg_acc += acc  
        avg_f1 += f1  
        avg_recall += recall  
        avg_mcc += mcc
        
        results = results.append({'AUC': auc, 'Accuracy': acc, 'F1': f1, 'Recall': recall, 'MCC': mcc}, ignore_index=True) 
results.to_csv('测试集预测_results.csv', index=False)

avg_auc /= len(range(start_index, end_index)) 
avg_acc /= len(range(start_index, end_index)) 
avg_f1 /= len(range(start_index, end_index))  
avg_recall /= len(range(start_index, end_index))
avg_mcc /= len(range(start_index, end_index))

auc_te_values = results['AUC'].values
accuracy_te_values = results['Accuracy'].values
f1_te_values = results['F1'].values
recall_te_values = results['Recall'].values
mcc_te_values = results['MCC'].values

std_auc_te = np.std(auc_te_values)
std_acc_te = np.std(accuracy_te_values)
std_f1_te = np.std(f1_te_values)
std_recall_te = np.std(recall_te_values)
std_mcc_te = np.std(mcc_te_values)

print(f'Average AUC: {avg_auc:.6f}±{std_auc_te:.6f}')
print(f'Average Accuracy: {avg_acc:.6f}±{std_acc_te:.6f}')
print(f'Average F1: {avg_f1:.6f}±{std_f1_te:.6f}')
print(f'Average Recall: {avg_recall:.6f}±{std_recall_te:.6f}')
print(f'Average MCC: {avg_mcc:.6f}±{std_mcc_te:.6f}')

                                                SMILES  active         0    1  \
0    CC1(C)N(OS(=O)(=O)O)C(=O)[C@H]1NC/C(=N/OCC(=O)...     0.0  0.016417  0.0   
1    NCC[C@@H](O)C(=O)N[C@@H]1C[C@H](N)[C@H](O[C@@H...     0.0  0.004457  0.0   
2                 CCC(=O)C(=O)c1ccc(OCC(=O)O)c(Cl)c1Cl     1.0  0.054507  0.0   
3    CCCCC(=O)O[C@H]1CC[C@H]2[C@@H]3CCc4cc(O)ccc4[C...     0.0  0.321503  0.0   
4                                          CC(N)C(=O)O     1.0  0.000000  0.0   
..                                                 ...     ...       ...  ...   
124  COc1ccc2c3c1O[C@@H]1C(=O)CC[C@]4(O)[C@H](C2)N(...     1.0  0.355105  0.0   
125            Nc1nc(NCCc2ccc(O)cc2)nc2nc(c3ccco3)nn12     0.0  0.067366  0.0   
126  CCCCc1nc(C)c(CC(=S)N(C)C)c(=O)n1Cc1ccc(c2ccccc...     0.0  0.125567  0.0   
127  CC(=O)[C@H]1CC[C@H]2[C@@H]3CC=C4C[C@@H](OC(=O)...     1.0  0.338337  0.0   
128            C[C@H](C(=O)O)c1ccc(N2Cc3ccccc3C2=O)cc1     1.0  0.175397  0.0   

            2         3    

                                                SMILES  active         0    1  \
0    COc1cccc(OC)c1C(=O)N[C@@H]1C(=O)N2[C@@H](C(=O)...       0  0.000000  0.0   
1    CCC[C@H]1O[C@H]2C[C@H]3[C@@H]4C[C@H](F)C5=CC(=...       1  0.311698  0.0   
2    O=S(=O)(c1ccc(Cl)cc1)[C@@H]1[C@H](CO)[C@H]1c1c...       1  0.110724  0.0   
3    COc1cccc2C(=O)c3c(O)c4C[C@@](O)(C[C@@H](O[C@H]...       0  0.050428  0.0   
4                              Brc1cccc(Br)c1N=C1NCCN1       1  0.086606  0.0   
..                                                 ...     ...       ...  ...   
124  COc1cc(C2c3cc4c(cc3C(OC3OC5COC(C)OC5C(O)C3O)C3...       0  0.065901  0.0   
125         COc1cc(N)c(Cl)cc1C(=O)NC1CCN(Cc2ccccc2)CC1       1  0.137272  0.0   
126                                     FC(Br)C(F)(F)F       1  0.234055  0.0   
127   CC(C)n1cc2CC3C(CC(CN3C)C(=O)NC3CCCCC3)c3cccc1c23       1  0.300840  0.0   
128  c1(ccc(c(c1)Cl)Cl)CC(N1C(CC2(CC1)NC(NC2=O)=O)C...       0  0.229174  0.0   

            2         3    

In [18]:
hidden_dim1 = 512
hidden_dim2 = 256
hidden_dim3 = 64
learning_rate = 0.1
num_epochs = 100
weight_decay = 0.001

Average AUC: 0.850873±0.116283
Average Accuracy: 0.658915±0.239119
Average F1: 0.647554±0.376343
Average Recall: 0.730789±0.422637
Average MCC: 0.383128±0.270336




    hidden_dim1 = 512
    hidden_dim2 = 256
    hidden_dim3 = 64
learning_rate = 0.1
num_epochs = 100
weight_decay = 0.01

Average AUC: 0.914110±0.023841
Average Accuracy: 0.451550±0.226031
Average F1: 0.224390±0.388655
Average Recall: 0.250000±0.433013
Average MCC: 0.148338±0.256930

    

    hidden_dim1 = 512
    hidden_dim2 = 256
    hidden_dim3 = 64
learning_rate = 0.01
num_epochs = 100
weight_decay = 0.01

Average AUC: 0.943448±0.020126
Average Accuracy: 0.796512±0.098723
Average F1: 0.831558±0.094451
Average Recall: 0.774691±0.173856
Average MCC: 0.624705±0.130822





    output_dim = 1
    hidden_dim1 = 512
    hidden_dim2 = 256
    hidden_dim3 = 64
    learning_rate = 0.001
    num_epochs = 100
    weight_decay = 0.01
    
Average AUC: 0.725601±0.064932
Average Accuracy: 0.598296±0.056470
Average F1: 0.666462±0.026149
Average Recall: 0.785408±0.030126
Average MCC: 0.353964±0.151104

SyntaxError: invalid syntax (3509277829.py, line 8)